In [45]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline
import statsmodels.formula.api as smf 
import statsmodels.api as sm
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier


***Question1***

Predicting flight delays using a dataset from the Bureau of Transportation Statistics

Variables: 
    1. MONTH
    2. DAY_OF_WEEK
    3. FL_DATE (flight date)
    4. UNIQUE_CARRIER
    5. FL_NUM (flight number)
    6. ORIGIN (airport code)
    7. ORIGIN_CITY_NAME
    8. DEST (airport code)
    9. DEST_CITY_NAME
    10. CRS_DEP_TIME (departure time)
    11. ARR_DEL15 (arrival delay greater than 15 minutes — the target)
    12. CRS_ELAPSED_TIME
    13. DISTANCE (miles between origin and destination)|

Given the independent variables above, I include only several variables for the remainder of my model in which I believe will deliver purposeful impact during prediction in this classification task. Conversely, I omit such variables in which I believe do not/will not serve as good predictors in the classification task. 

Included Independent variables:
1. MONTH
2. DAY_OF_WEEK
3. FL_DATE (flight date)
5. FL_NUM (flight number)
6. ORIGIN (airport code)
8. DEST (airport code)
10. CRS_DEP_TIME (departure time)
12. CRS_ELAPSED_TIME
13. DISTANCE (miles between origin and destination)|

Excluded variables:
7. ORIGIN_CITY_NAME
9. DEST_CITY_NAME

The decision to exclude origin and destination city stems from the fact that it represents duplicate information when ORIGIN(airport code) and DEST(airport code) has already been taken into account in the model. 

This notebook will continue as follows: 
1. Data cleaning 
   - Clean data to only include relevant variables 
   - Check for any NaN values in the dataset. If they exist, replace the NaN cell with the most frequently observed value in the respective column using scikit-learn's Imputer. 
   - Convert String variables to numerical values using scikit-learn's Label Encoder.
   - Consolidate dataset
   
2. Classification task
   - Random Forest vs AdaBoost vs XG Boost 
   - results 
   - Conclusion & Discussion 

***1. Data Observation***

In [27]:
os.getcwd()
flightdata = pd.read_csv('/Users/Rong/Documents/USF/Machine Learning 2/MidTerm2/aggregated.csv')
print(flightdata.head())

   MONTH  DAY_OF_WEEK     FL_DATE UNIQUE_CARRIER  FL_NUM ORIGIN  \
0    2.0          6.0  2017-02-25             B6    28.0    MCO   
1    2.0          7.0  2017-02-26             B6    28.0    MCO   
2    2.0          1.0  2017-02-27             B6    28.0    MCO   
3    2.0          2.0  2017-02-28             B6    28.0    MCO   
4    2.0          3.0  2017-02-01             B6    33.0    BTV   

  ORIGIN_CITY_NAME DEST DEST_CITY_NAME  CRS_DEP_TIME  ARR_DEL15  \
0      Orlando, FL  EWR     Newark, NJ        1000.0        0.0   
1      Orlando, FL  EWR     Newark, NJ         739.0        0.0   
2      Orlando, FL  EWR     Newark, NJ        1028.0        0.0   
3      Orlando, FL  EWR     Newark, NJ         739.0        0.0   
4   Burlington, VT  JFK   New York, NY        1907.0        0.0   

   CRS_ELAPSED_TIME  DISTANCE  Unnamed: 13  
0             156.0     937.0          NaN  
1             153.0     937.0          NaN  
2             158.0     937.0          NaN  
3             

***Checking data and cleaning***

Due to the duplicity of ORIGIN_CITY_NAME and DEST_CITY_NAME, I will drop these two columns from the dataset. According to the analysis by the Bereau of Transportation Statistics, the most relevant information regarding the complex organisation of flights is 'Airport code'. This is due to the fact that there could be multiple airports within the same city. As such, I conjecture that the city name will not deliver any additional information gain for the purposes of this classification task. 

In [28]:
data = flightdata.iloc[:,:-1]
data = data.drop(['ORIGIN_CITY_NAME', 'DEST_CITY_NAME'], axis =1)
print(data.head())

   MONTH  DAY_OF_WEEK     FL_DATE UNIQUE_CARRIER  FL_NUM ORIGIN DEST  \
0    2.0          6.0  2017-02-25             B6    28.0    MCO  EWR   
1    2.0          7.0  2017-02-26             B6    28.0    MCO  EWR   
2    2.0          1.0  2017-02-27             B6    28.0    MCO  EWR   
3    2.0          2.0  2017-02-28             B6    28.0    MCO  EWR   
4    2.0          3.0  2017-02-01             B6    33.0    BTV  JFK   

   CRS_DEP_TIME  ARR_DEL15  CRS_ELAPSED_TIME  DISTANCE  
0        1000.0        0.0             156.0     937.0  
1         739.0        0.0             153.0     937.0  
2        1028.0        0.0             158.0     937.0  
3         739.0        0.0             153.0     937.0  
4        1907.0        0.0              90.0     266.0  


***Checking for null values***

Operation to check if there are any Null values int the dataset provided. 

If return True - Null values exist and we search for columns with null values

If return False - Null values do not exist.

In [29]:
print('Null values exists? --> ' +  str(data.isnull().values.any()))

Null values exists? --> True


In [30]:
count = -1
for i in data.columns:
    count = count + 1
    if data[i].isnull().values.any() == True:
        print(i)
        print(count)

ARR_DEL15
8
CRS_ELAPSED_TIME
9


***Data cleaning***

We extract values the columns with NaN values and fill in the cells using the Scikit-learn' Imputer. 

For the purposes of this model, we use 'most frequent' as the value to be filled. 



In [31]:
#Extract Y from data 
Y = data.iloc[:, 8]
Y = Y.to_frame(name=None)

In [32]:
#Extract crs_elapsed_time from X
crs_elapsed_time = data.iloc[:, 9]
crs_elapsed_time = crs_elapsed_time.to_frame(name=None)

In [33]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values = 'NaN', strategy = 'most_frequent', axis = 0)

Y_imputed = imputer.fit_transform(Y)
crs_elapsed_time_imputed = imputer.fit_transform(crs_elapsed_time)

In [37]:
#Place crs_elapsed_time back into X 
data.iloc[:, 8] = Y_imputed
data.iloc[:, 9] = crs_elapsed_time_imputed

In [38]:
def encode_labels(labels):
    
    from sklearn import preprocessing 
    
    le = preprocessing.LabelEncoder()
    
    return le.fit_transform(labels)

***We convert columns with String formats into numerical values*** 

This is purely for the purpose of modelling

In [40]:
#Label encode selected X columns 
fl_date = data.loc[:, 'FL_DATE']
unique_carrier = data.loc[:,'UNIQUE_CARRIER']
origin = data.loc[:,'ORIGIN']
dest = data.loc[:,'DEST']

fl_date_lbl = encode_labels(fl_date)
unique_carrier_lbl = encode_labels(unique_carrier)
origin_lbl = encode_labels(origin)
dest_lbl = encode_labels(dest)

data.loc[:, 'FL_DATE'] = fl_date_lbl
data.loc[:,'UNIQUE_CARRIER'] = unique_carrier_lbl
data.loc[:,'ORIGIN'] = origin_lbl
data.loc[:,'DEST'] = dest_lbl
print(data.head())

   MONTH  DAY_OF_WEEK  FL_DATE  UNIQUE_CARRIER  FL_NUM  ORIGIN  DEST  \
0    2.0          6.0      330               2    28.0     188    98   
1    2.0          7.0      331               2    28.0     188    98   
2    2.0          1.0      332               2    28.0     188    98   
3    2.0          2.0      333               2    28.0     188    98   
4    2.0          3.0      306               2    33.0      48   158   

   CRS_DEP_TIME  ARR_DEL15  CRS_ELAPSED_TIME  DISTANCE  
0        1000.0        0.0             156.0     937.0  
1         739.0        0.0             153.0     937.0  
2        1028.0        0.0             158.0     937.0  
3         739.0        0.0             153.0     937.0  
4        1907.0        0.0              90.0     266.0  


***Finalise X and Y to be used in the learning process***

In [51]:
#Finalise data
ready_X = data.drop(['ARR_DEL15'], axis =1)
# print(ready_X.head())
ready_Y = data.iloc[:,8]
# print(ready_Y)

***2. Classification Task***

I have decided to use Random Forest and compare its results against XGBoost and AdaBoost for several reasons. 

**Random Forest**

Firstly, due to the robustness of the Random Forest algorithm, I believe this serves as a good first shot attempt at predicting flight delays. Further, given the vastness of the dataset, I believe an ensemble method will be robust since we will be able to lower the variance in the data due to the bagging characteristic of this algorithm. This also prevents overfit. 

**Adaboost**

I then use Adaboost in an attempt to boost the robustness of the results in Random Forest. As the 'weak learners' is combined into a weighted sum, this will aid the final output of the boosted classifier - subsequent weak learners are tweaked in favor of those instances misclassified by previous classifiers and so given higher preference. Finally, I use this as it will be less susceptible to the overfitting problem in comparision to Random Forest in singleton.

**XGBoost**

This is used to contrast with Adaboost. 

***Random Forest***

In [334]:
clf1 = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', max_features = 'auto', max_depth = 6, bootstrap = True, random_state = 1)
cv_rf = cross_validate(clf1, X=ready_X, y=ready_Y, cv=10, scoring=['accuracy', 'recall', 'roc_auc', 'precision'])
#Place into dataframe
cv_rf_df = pd.DataFrame(cv_rf)
cv_rf_df

/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Rong/anaconda3/lib/python3.6/site-packages/sklear

***XGBoost***


In [336]:
clf2 = XGBClassifier(n_estimators=10, max_depth=6, learning_rate=0.1, subsample=0.5, random_state=1)
cv_xg = cross_validate(clf2, X=ready_X, y=ready_Y, cv=10, scoring=['accuracy', 'recall', 'roc_auc', 'precision'])
#Place into dataframe
cv_xg_df = pd.DataFrame(cv_xg)
cv_xg_df


/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_precision'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_recall'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/Rong/anaconda3/lib/python3.6/site-packag

,fit_time,score_time,test_accuracy,test_precision,test_recall,test_roc_auc,train_accuracy,train_precision,train_recall,train_roc_auc
0,47.263188,1.578935,0.822380,0.000000,0.000000,0.489290,0.822737,0.720096,0.003282,0.676369
1,45.621868,1.572849,0.723743,0.051359,0.031787,0.459767,0.824119,1.000000,0.009787,0.671549
2,44.828410,1.478678,0.574097,0.113976,0.206362,0.427419,0.828615,0.933038,0.037810,0.678065
3,47.085224,1.540048,0.701412,0.037865,0.027901,0.300049,0.824196,0.936283,0.010968,0.682720
4,45.941356,1.442077,0.822381,0.000000,0.000000,0.321226,0.822381,0.000000,0.000000,0.673167
5,49.019903,1.541995,0.759404,0.000000,0.000000,0.599130,0.826236,0.992473,0.021870,0.666032
6,48.929838,1.613355,0.815920,0.340305,0.038757,0.606808,0.822415,1.000000,0.000191,0.670670
7,46.867644,1.608480,0.822381,0.000000,0.000000,0.649863,0.822381,0.000000,0.000000,0.666794
8,47.943345,1.544320,0.822381,0.000000,0.000000,0.604839,0.822535,0.745003,0.001318,0.672980
9,45.742939,1.715453,0.313133,0.205462,1.000000,0.582828,0.823809,1.000000,0.008039,0.660594


***AdaBoost***

In [338]:
clf3 = AdaBoostClassifier(n_estimators=10, learning_rate= 0.1, random_state=1)
cv_ab = cross_validate(clf3, X=ready_X, y=ready_Y, cv=10, scoring=['accuracy', 'recall', 'roc_auc', 'precision'])
#Place into dataframe
cv_ab_df = pd.DataFrame(cv_ab)
cv_ab_df


/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Rong/anaconda3/lib/python3.6/site-packages/sklear

***Evaluation***

The results delivered by all three classifiers show strength in the test_accuracy with an average above 0.80, however, the continous upset in precision and recall leads me to believe that the results may be continously skewed. 

Precision is the fraction of relevant instances among the retrieved instances.

Recall is the fraction of relevant instances that have been retrieved over the total amount of relevant instances.

In general we want to aim for high precision and high recall since Precision tell us how useful the search results are and Recall is how complete the results are. However, the consistent values edging around the 0 mark makes me believe that:

1. Precision the algorithm did not return any substantially relevant results than irrelevant ones

2. Recall means that an algorithm returned none of the relevant results.

Further analysis finds that there is huge imbalance of delayed instances when compared to not-delayed instances. Specifically it is 1:4 at 0.21%. 


As such the approach I take is to curate a balanced dataset, approximately in the ratio of 1:1 and re-run this data. This should adjust to deliver more accurate results in predicting flight delays. 

***Observing the imbalance in the dataset***

In [52]:
delayed = ready_Y[ready_Y == 1]
notdelayed = ready_Y[ready_Y == 0]
print(len(delayed))
print(len(notdelayed))
print(len(delayed)/len(notdelayed))

911071
4218283
0.2159814787201333


***Creating a balanced dataset***

In [47]:
delayed = data[data['ARR_DEL15'] == 1]
notdelayed = data[data['ARR_DEL15'] == 0]

notdelayed = notdelayed.sample(n=911071, replace=False, random_state=1)
frames = [delayed, notdelayed]
data_adjusted = pd.concat(frames)

new_X = data_adjusted.drop(['ARR_DEL15'], axis =1)
new_Y = data_adjusted.loc[:,'ARR_DEL15']

***Running Random Forest, AdaBoost and XGBoost with the datasert***

***Random Forest***

In [48]:
clf4 = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', max_features = 'auto', max_depth = 6, bootstrap = True, random_state = 1)
cv_rf_new = cross_validate(clf4, X=new_X, y=new_Y, cv=10, scoring=['accuracy', 'recall', 'roc_auc', 'precision'])
cv_rf_df_new = pd.DataFrame(cv_rf_new)
print(cv_rf_df_new)

    fit_time  score_time  test_accuracy  test_precision  test_recall  \
0  27.215166    0.686914       0.295720        0.000644     0.000263   
1  25.282983    0.692719       0.294818        0.009396     0.003929   
2  26.106565    0.678356       0.346999        0.200116     0.102100   
3  24.940020    0.669907       0.296388        0.028325     0.012227   
4  25.624852    0.677847       0.297299        0.041192     0.018198   
5  24.743069    0.629626       0.447260        0.437825     0.371387   
6  25.256049    0.675231       0.365768        0.263283     0.149297   
7  24.948683    0.854897       0.296624        0.001480     0.000604   
8  25.011014    0.712125       0.319119        0.109500     0.050721   
9  25.776631    0.660226       0.291449        0.000000     0.000000   

   test_roc_auc  train_accuracy  train_precision  train_recall  train_roc_auc  
0      0.100928        0.640439         0.628081      0.688682       0.698949  
1      0.144867        0.624719         0.61622

/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_precision'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_recall'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/Rong/anaconda3/lib/python3.6/site-packag

***XGBoost***

In [49]:
clf5 = XGBClassifier(n_estimators=10, max_depth=6, learning_rate=0.1, subsample=0.5)
cv_xg_new = cross_validate(clf5, X=new_X, y=new_Y, cv=10, scoring=['accuracy', 'recall', 'roc_auc', 'precision'])
cv_xg_df_new = pd.DataFrame(cv_xg_new)
print(cv_xg_df_new)


    fit_time  score_time  test_accuracy  test_precision  test_recall  \
0  27.796455    0.590970       0.291292        0.000000     0.000000   
1  29.469010    0.590980       0.301228        0.042539     0.018484   
2  26.577532    0.681694       0.346099        0.176111     0.083682   
3  27.913726    0.560564       0.293841        0.008736     0.003666   
4  29.179602    0.734634       0.292985        0.029910     0.013171   
5  30.072061    0.677029       0.356592        0.248416     0.141603   
6  28.792837    0.664532       0.376041        0.291821     0.173763   
7  30.588181    0.662444       0.312232        0.053971     0.022721   
8  28.356149    0.620402       0.314460        0.108178     0.051225   
9  28.596966    0.669398       0.284753        0.000280     0.000121   

   test_roc_auc  train_accuracy  train_precision  train_recall  train_roc_auc  
0      0.082561        0.643737         0.628510      0.702981       0.703116  
1      0.099527        0.637010         0.62412

/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_precision'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_recall'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/Rong/anaconda3/lib/python3.6/site-packag

***AdaBoost***

In [53]:
clf6 = AdaBoostClassifier(n_estimators=10, learning_rate= 0.1)
cv_ab_new = cross_validate(clf6, X=new_X, y=new_Y, cv=10, scoring=['accuracy', 'recall', 'roc_auc', 'precision'])
cv_ab_df_new = pd.DataFrame(cv_ab_new)
print(cv_ab_df_new)


    fit_time  score_time  test_accuracy  test_precision  test_recall  \
0  22.664850    1.008154       0.350183        0.275261     0.183497   
1  22.904869    0.828423       0.292047        0.000000     0.000000   
2  21.729513    0.823942       0.418568        0.394584     0.304806   
3  19.331246    0.818641       0.286822        0.000000     0.000000   
4  23.310710    0.838609       0.578177        0.567672     0.655800   
5  25.725943    0.791487       0.605277        0.582643     0.742215   
6  19.702431    0.710503       0.599098        0.579773     0.720219   
7  21.279591    0.752411       0.578973        0.566110     0.676260   
8  21.712049    0.763443       0.555457        0.550604     0.603400   
9  21.145820    0.755412       0.257450        0.000000     0.000000   

   test_roc_auc  train_accuracy  train_precision  train_recall  train_roc_auc  
0      0.207474        0.608826         0.591814      0.701469       0.639188  
1      0.311898        0.606192         0.60195

/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_precision'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/Rong/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_recall'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/Rong/anaconda3/lib/python3.6/site-packag

***Conclusion***


Overall, although adjusting the dataset decreases the prediction accuracy score significantly. Specifically by 50%. I believe these results are more representative of the dataset and thus giving the reader a more representative and holistic view in predicting flight delays.  

In [56]:
print(cv_ab_df_new['test_accuracy'].mean())


0.45220526393877647
